# EXAMPLE 1: Correct and display

This example covers the basic usage of the dual-PRF velocity correction function:

- [Load raw data with Py-ART](#load_data_pyart)  
- [Apply the correction function](#apply_vcor)  
- [Display the results](#display)


**EVENT**: A tornado associated to a rotating cell that took place near Cardona town (41º54'51'' N, 1º40'52'' E) on the 7th of January 2018.

**DATA**: It works with data from the weather radar network (XRAD) of the Meteorological Service of Catalonia. This data is in IRIS RAW format. In this particular case, the data used is from Creu del Vent (CDV) radar.

**LIBRARIES/FUNCTIONS:**

In [ ]:
import pyart
import vcor_dual_prf as vcor

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

**RADAR SETTINGS:**

Input file, elevation/sweep number

In [ ]:
file_in = '../sample_data/CDV180107_tornado.RAW'
sw = 3

**DISPLAY SETTINGS**

Site coordinates (decimal lat/lon), display region limits (decimal lat/lon), tick locations

In [ ]:
ylim = (-40, 60)
xlim = (-25, 75)
range_rings = range(25, 150, 25)

Set a custom colormap for the velocity

In [ ]:
cmaplst_l = [plt.get_cmap('ocean', 22)(i) for i in list(range(1, 22))] + [plt.get_cmap('gnuplot_r', 30)(i) for i in list(range(1, 21))]
cmap_vel = mpl.colors.ListedColormap(cmaplst_l)
cmap_vel.set_bad('lightgrey', 1.)

<a id='load_data_pyart'></a>
### Load raw data with Py-ART

Py-ART reads the data and metadata in the input file into a dictionary structure (radar object instance):

In [ ]:
rad = pyart.io.read(file_in)

Let's check the data fields that have been loaded:

In [ ]:
rad.fields.keys()

For plotting purposes only, we retrieve the Nyquist velocity stored in the radar instance metadata :

In [ ]:
# Nyquist velocity
v_ny = rad.instrument_parameters['nyquist_velocity']['data'][0]
v_ny

<a id='apply_vcor'></a>
### Apply the correction function

As a first example, we apply the correction function for all 4 the methods available. 

The function parameters specified in each case correspond (as precisely as possible) to the ones detailed in the original publications. When a parameter is not explicitly mentioned in the publication, we leave the one set by default in the function. 

Application of the function includes the addition of a new data field in the radar instance. 


**(a) 'mean' method**: 

Joe, P. and May, P. T., 2003: Correction of dual PRF velocity errors for operational Doppler weather radars. *J. Atmos. Oceanic Technol.*, 20(4), 429-442

In [ ]:
vcor.correct_dualprf(radar=rad, two_step=False, 
                     method_det='mean', kernel_det=np.ones((3, 3)),
                     vel_field='velocity', new_field='vcor_mean')

**(b) 'median' method**:

Holleman, I. and Beekhuis, H., 2003: Analysis and correction of dual PRF velocity data. *J. Atmos. Oceanic Technol.*, 20(4), 443-453 

In [ ]:
vcor.correct_dualprf(radar=rad, two_step=False, 
                     method_det='median', kernel_det=np.ones((3, 3)), 
                     vel_field='velocity', new_field='vcor_median')

**(c) 'cmean_sc' method**: 

Altube, P., et al., 2017: Correction of dual-PRF Doppler velocity outliers in the presence of aliasing. *J. Atmos. Oceanic Technol.*, 34(7), 1529-1543 

In [ ]:
vcor.correct_dualprf(radar=rad, two_step=True, 
                     method_det='cmean_sc', kernel_det=np.ones((7, 7)),
                     method_cor='median', kernel_cor=np.ones((7, 7)),
                     vel_field='velocity', new_field='vcor_cmean_sc')

**(d) 'cmean' method**: Hengstebeck, T., et al., 2018: Radar network–based detection of mesocyclones at the German
Weather Service. *J. Atmos. Oceanic Technol.*, 35(2), 299-321

In [ ]:
vcor.correct_dualprf(radar=rad, two_step=True,
                     method_det='cmean', kernel_det=np.ones((11, 11)),
                     method_cor='cmean', kernel_cor=np.ones((5, 5)),
                     vel_field='velocity', new_field='vcor_cmean')

Let's check that all the corrections have been stored in the radar instance:

In [ ]:
rad.fields.keys()

<a id='display'></a>
### Display the results

In [ ]:
display = pyart.graph.RadarDisplay(rad)
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111)

display.plot('velocity', sw, ax=ax, vmin=-v_ny, vmax=v_ny, mask_outside=False, 
             cmap=cmap_vel, colorbar_flag=True, title_flag=False)
display.plot_range_rings(range_rings, lw=0.8, ls=':', ax=ax)
display.plot_cross_hair(0.5, ax=ax)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18,15))

display.plot('vcor_mean', sw, ax=ax1, vmin=-v_ny, vmax=v_ny, mask_outside=False, 
             cmap=cmap_vel, colorbar_flag=True, title_flag=False)
display.plot_range_rings(range_rings, lw=0.8, ls=':', ax=ax1)
display.plot_cross_hair(0.5, ax=ax1)
ax1.set_xlim(xlim)
ax1.set_ylim(ylim)

display.plot('vcor_median', sw, ax=ax2, vmin=-v_ny, vmax=v_ny, mask_outside=False, 
            cmap=cmap_vel, colorbar_flag=True, title_flag=False)
display.plot_range_rings(range_rings, lw=0.8, ls=':', ax=ax2)
display.plot_cross_hair(0.5, ax=ax2)
ax2.set_xlim(xlim)
ax2.set_ylim(ylim)

display.plot('vcor_cmean_sc', sw, ax=ax3, vmin=-v_ny, vmax=v_ny, mask_outside=False, 
            cmap=cmap_vel, colorbar_flag=True, title_flag=False)
display.plot_range_rings(range_rings, lw=0.8, ls=':', ax=ax3)
display.plot_cross_hair(0.5, ax=ax3)
ax3.set_xlim(xlim)
ax3.set_ylim(ylim)

display.plot('vcor_cmean', sw, ax=ax4, vmin=-v_ny, vmax=v_ny, mask_outside=False, 
            cmap=cmap_vel, colorbar_flag=True, title_flag=False)
display.plot_range_rings(range_rings, lw=0.8, ls=':', ax=ax4)
display.plot_cross_hair(0.5, ax=ax4)
ax4.set_xlim(xlim)
ax4.set_ylim(ylim)

fig.tight_layout()
plt.show()